In [ ]:
!pip install -U git+https://github.com/Sakib323/AI-Game-Engine.git
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers
!pip install triton==3.2.0
!pip install datasets
!pip install wandb
!pip install -U datasets
!pip install triton==2.3.1
!pip install objaverse
!pip install diffusers
!pip install trimesh
!pip install jaxtyping
!pip install pytorch-lightning
!pip install ijson

  Cloning https://github.com/Sakib323/AI-Game-Engine.git to /tmp/pip-req-build-cnfrn4yh
  Running command git clone --filter=blob:none --quiet https://github.com/Sakib323/AI-Game-Engine.git /tmp/pip-req-build-cnfrn4yh
  Resolved https://github.com/Sakib323/AI-Game-Engine.git to commit 505b838f2351b38c9fd462281106d55ad311783c
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:

In [ ]:
!git clone --depth 1 --branch main https://github.com/stepfun-ai/Step1X-3D.git
import sys
sys.path.append("/content/Step1X-3D")
import os
print(os.listdir("/content/Step1X-3D"))
!pip install -r /content/Step1X-3D/requirements.txt --verbose

Streaming output truncated to the last 5000 lines.
          Alternatively, you can also rely on setuptools' discovery methods
          (for example by using `find_namespace_packages(...)`/`find_namespace:`
          instead of `find_packages(...)`/`find:`).

          You can read more about "package discovery" on setuptools documentation page:

          - https://setuptools.pypa.io/en/latest/userguide/package_discovery.html

          If you don't want 'deepspeed.ops.csrc.spatial.includes' to be distributed and are
          already explicitly excluding 'deepspeed.ops.csrc.spatial.includes' via
          `find_namespace_packages(...)/find_namespace` or `find_packages(...)/find`,
          you can try to use `exclude_package_data`, or `include-package-data=False` in
          combination with a more fine grained `package-data` configuration.

          You can read more about "package data files" on setuptools documentation page:

          - https://setuptools.pypa.io/en/latest/use

In [ ]:
pip install torch-cluster -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
  Using cached https://data.pyg.org/whl/torch-2.6.0%2Bcu124/torch_cluster-1.6.3%2Bpt26cu124-cp311-cp311-linux_x86_64.whl (3.4 MB)


3D OBJECT CAPTIONED Dataset

In [ ]:
import requests
import gzip
import json
import objaverse
import gc
import os
import ijson


KEYS_TO_EXCLUDE = {
    "tags", "name", "staffpickedAt", "viewCount", "likeCount", "animationCount",
    "commentCount", "publishedAt", "user", "description", "faceCount", "createdAt",
    "vertexCount", "license", "uri", "viewerUrl", "embedUrl", "isDownloadable",
    "categories", "isAgeRestricted", "archives"
}

print("Starting dataset preparation...")

# Check if captions file exists; download only if necessary
url = "https://huggingface.co/datasets/tiange/Cap3D/resolve/main/Objaverse_files/cap3d_captions.json.gz"
captions_file = 'cap3d_captions.json.gz'
if not os.path.exists(captions_file):
    print("Downloading captions file...")
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(captions_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Captions file downloaded successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading captions: {e}")
        exit()
else:
    print("Captions file already exists, skipping download.")

def process_batch(batch, f, total_counter):
    uids = [uid for uid, _ in batch]
    try:
        annotations = objaverse.load_annotations(uids)
    except Exception as e:
        print(f"Error loading annotations for batch: {e}")
        return 0  # Skip this batch on error

    for i, (uid, caption) in enumerate(batch):
        metadata = annotations.get(uid, {})
        filtered_metadata = {k: v for k, v in metadata.items() if k not in KEYS_TO_EXCLUDE}

        # Filter thumbnails to keep only 1024x576 images
        if "thumbnails" in filtered_metadata and "images" in filtered_metadata["thumbnails"]:
            filtered_metadata["thumbnails"]["images"] = [
                img for img in filtered_metadata["thumbnails"]["images"]
                if img.get("width") == 1024 and img.get("height") == 576
            ]

        datapoint = {
            "uid": uid,
            "caption": caption,
            **filtered_metadata
        }
        f.write(json.dumps(datapoint) + '\n')
        if total_counter + i < 10:
            print(f"Datapoint {uid}:")
            print(json.dumps(datapoint, indent=2))
            print("-" * 50)
    del annotations
    gc.collect()
    return len(batch)


batch_size = 100000
total_counter = 0
with open('extended_dataset.jsonl', 'w', encoding='utf-8') as f:
    batch = []
    for uid, caption in ijson.kvitems(gzip.open(captions_file, 'rt', encoding='utf-8'), ''):
        batch.append((uid, caption))
        if len(batch) >= batch_size:
            processed = process_batch(batch, f, total_counter)
            total_counter += processed
            print(f"Processed {total_counter} captions so far.")
            batch = []
    if batch:
        processed = process_batch(batch, f, total_counter)
        total_counter += processed

print(f"Total entries in extended dataset: {total_counter}")
print("Dataset saved to 'extended_dataset.jsonl'")
print("Script finished.")

Starting dataset preparation...
Captions file downloaded successfully.


 99%|█████████▉| 159/160 [01:57<00:00,  1.35it/s]


Datapoint ed51a51909ee46c780db3a85e821feb2:
{
  "uid": "ed51a51909ee46c780db3a85e821feb2",
  "caption": "Matte green rifle with a long barrel, stock, and detailed magazine.",
  "thumbnails": {
    "images": [
      {
        "uid": "0d6d45c0ee174cca9323a183ebbe2ef3",
        "size": 10719,
        "width": 1024,
        "url": "https://media.sketchfab.com/models/ed51a51909ee46c780db3a85e821feb2/thumbnails/f46b00b385d4449d923aff78348b04c6/eb41be4a1dcd450a974db9d24dc13d16.jpeg",
        "height": 576
      }
    ]
  }
}
--------------------------------------------------
Datapoint 9110b606f6c547b2980fcb3c8c4b6a1c:
{
  "uid": "9110b606f6c547b2980fcb3c8c4b6a1c",
  "caption": "Rustic single-story building with a weathered green gable roof, exposed wooden beams, brick walls, and a partly visible glass window front.",
  "thumbnails": {
    "images": [
      {
        "uid": "32b5631f607543a38e10188b03e3d7c1",
        "size": 80170,
        "width": 1024,
        "url": "https://media.sketchfab

 99%|█████████▉| 159/160 [01:07<00:00,  2.37it/s]


Processed 200000 captions so far.


 99%|█████████▉| 159/160 [01:06<00:00,  2.40it/s]


Processed 300000 captions so far.


  6%|▌         | 9/160 [00:03<00:56,  2.65it/s]


KeyboardInterrupt: 

In [ ]:
import json
import objaverse
import pathlib
import shutil
import os

jsonl_path = "extended_dataset.jsonl"
output_dir = pathlib.Path("downloads")
output_dir.mkdir(exist_ok=True)
metadata_output_path = output_dir / "metadata.json"
download_limit = 100

uid_metadata = {}
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        if len(uid_metadata) >= download_limit:
            break
        entry = json.loads(line)
        uid = entry["uid"]
        caption = entry.get("caption", "")
        thumbnails = entry.get("thumbnails", {}).get("images", [])
        thumbnail_urls = [img["url"] for img in thumbnails if "url" in img]

        uid_metadata[uid] = {
            "caption": caption,
            "thumbnails": thumbnail_urls
        }

uids = list(uid_metadata.keys())
print(f"Downloading {len(uids)} models...")
paths = objaverse.load_objects(uids=uids)

final_metadata = {}
for uid, src_path in paths.items():
    dst_path = output_dir / f"{uid}.glb"
    try:
        shutil.copy(src_path, dst_path)
        final_metadata[uid] = uid_metadata[uid]
        print(f"Saved {uid} to {dst_path}")
    except Exception as e:
        print(f"Failed to save {uid}: {e}")

with open(metadata_output_path, "w", encoding="utf-8") as f:
    json.dump(final_metadata, f, indent=2)

print(f"\nDownloaded {len(final_metadata)} models.")
print(f"Metadata saved to {metadata_output_path}")


Downloaded 1 / 100 objects
Downloaded 2 / 100 objects
Downloaded 3 / 100 objects
Downloaded 4 / 100 objects
Downloaded 5 / 100 objects
Downloaded 6 / 100 objects
Downloaded 7 / 100 objects
Downloaded 8 / 100 objects
Downloaded 9 / 100 objects
Downloaded 10 / 100 objects
Downloaded 11 / 100 objects
Downloaded 12 / 100 objects
Downloaded 13 / 100 objects
Downloaded 14 / 100 objects
Downloaded 15 / 100 objects
Downloaded 16 / 100 objects
Downloaded 17 / 100 objects
Downloaded 18 / 100 objects
Downloaded 19 / 100 objects
Downloaded 20 / 100 objects
Downloaded 21 / 100 objects
Downloaded 22 / 100 objects
Downloaded 23 / 100 objects
Downloaded 24 / 100 objects
Downloaded 25 / 100 objects
Downloaded 26 / 100 objects
Downloaded 27 / 100 objects
Downloaded 28 / 100 objects
Downloaded 29 / 100 objects
Downloaded 30 / 100 objects
Downloaded 31 / 100 objects
Downloaded 32 / 100 objects
Downloaded 33 / 100 objects
Downloaded 34 / 100 objects
Downloaded 35 / 100 objects
Downloaded 36 / 100 objects
D

In [ ]:
import requests
import os
import tempfile
import trimesh
import numpy as np
import torch
from transformers import AutoTokenizer
from tqdm import tqdm
import random
import traceback
import logging
import sys

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16 if device.type == "cuda" else torch.float32
print(f"Using device: {device}, dtype: {dtype}")

sys.path.append("/content/Step1X-3D")

from step1x3d_geometry.models.pipelines.pipeline import Step1X3DGeometryPipeline

from huggingface_hub import hf_hub_download
print("Initializing Step1X-3D VAE...")
try:
    geometry_pipeline = Step1X3DGeometryPipeline.from_pretrained(
        "stepfun-ai/Step1X-3D",
        subfolder='Step1X-3D-Geometry-1300m',
        torch_dtype=dtype
    )
    vae = geometry_pipeline.vae.to(device)
    vae.eval()
    print("Step1X-3D VAE initialized successfully.")
except Exception as e:
    print(f"Error initializing pipeline. Make sure you have the Step1X-3D repo cloned and in your sys.path. Error: {e}")
    vae = None

tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token


image_vae = AutoencoderKL.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        subfolder="vae",
        torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32
    ).to(DEVICE).eval()
    print("VAE model loaded.")

Using device: cuda, dtype: torch.float16
Initializing Step1X-3D VAE...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/766M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/5.27G [00:00<?, ?B/s]

...

Step1X-3D VAE initialized successfully.


tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:
from diffusers import AutoencoderKL
import requests
from PIL import Image
from io import BytesIO
import torch
import torchvision.transforms as transforms
from torchvision.transforms.functional import crop
from diffusers import AutoencoderKL
import torchvision.transforms as T


def get_image_latent(url: str, vae: AutoencoderKL, device: str = "cuda") -> torch.Tensor:

    try:
        response = requests.get(url)
        response.raise_for_status()
        img_data = BytesIO(response.content)
        raw_image = Image.open(img_data).convert("RGB")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return None
    except IOError as e:
        print(f"Error opening image: {e}")
        return None

    preprocess = T.Compose([
        T.Resize(512, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(512),
        T.ToTensor(),
        T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    image_tensor = preprocess(raw_image).unsqueeze(0).to(device)

    with torch.no_grad():
        latent_dist = vae.encode(image_tensor).latent_dist
        latents = latent_dist.sample()
        latents = latents * vae.config.scaling_factor

    return latents

In [ ]:
import os
import json
import torch
import traceback
import numpy as np
import trimesh
from tqdm import tqdm
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16 if device.type == "cuda" else torch.float32

def compute_dihedral_angle(normal1, normal2):
    cos_angle = np.dot(normal1, normal2)
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    return np.degrees(np.arccos(cos_angle))

def robust_sharp_edge_sampling(mesh, num_points=16384, sharp_threshold=30):
    uniform_count = int(num_points * 0.9)
    uniform_points, face_indices = trimesh.sample.sample_surface(mesh, uniform_count)
    uniform_normals = mesh.face_normals[face_indices]

    sharp_edges = []
    if hasattr(mesh, 'edges_face'):
        for edge, face_pair in zip(mesh.edges, mesh.edges_face):
            if isinstance(face_pair, (int, np.integer)):
                continue
            if len(face_pair) == 2:
                normal1 = mesh.face_normals[face_pair[0]]
                normal2 = mesh.face_normals[face_pair[1]]
                angle = compute_dihedral_angle(normal1, normal2)
                if angle > sharp_threshold:
                    sharp_edges.append(edge)

    sharp_count = num_points - uniform_count
    sharp_points, sharp_normals = [], []
    if sharp_edges:
        edges = mesh.vertices[np.array(sharp_edges)]
        for edge in edges:
            t = np.random.rand(sharp_count // max(1, len(sharp_edges)))
            points = (1 - t[:, None]) * edge[0] + t[:, None] * edge[1]
            sharp_points.extend(points)
            sharp_normals.extend([np.mean(mesh.face_normals[mesh.edges_face[mesh.edges.tolist().index(list(edge))]], axis=0)] * len(points))
    else:
        sharp_points, sharp_face_idx = trimesh.sample.sample_surface(mesh, sharp_count)
        sharp_normals = mesh.face_normals[sharp_face_idx]

    all_points = np.vstack([uniform_points, sharp_points])[:num_points]
    all_normals = np.vstack([uniform_normals, sharp_normals])[:num_points]
    return all_points, all_normals, np.array(sharp_points), np.array(sharp_normals)

def process_mesh_to_vae_input(mesh_path, num_points=32768):
    try:
        ext = os.path.splitext(mesh_path)[1].lower()
        mesh = trimesh.load(mesh_path, force='mesh', file_type=ext[1:] if ext else None)
        if isinstance(mesh, trimesh.Scene):
            mesh = mesh.dump().sum()
        if not mesh.is_watertight:
            try:
                mesh.fill_holes()
            except Exception:
                logger.warning(f"Hole filling failed for {mesh_path}")

        points, normals, sharp_points, sharp_normals = robust_sharp_edge_sampling(mesh, num_points=num_points)
        if points.shape[0] < num_points:
            pad = num_points - points.shape[0]
            points = np.pad(points, ((0, pad), (0, 0)), mode='wrap')
            normals = np.pad(normals, ((0, pad), (0, 0)), mode='wrap')
        if sharp_points.shape[0] < num_points:
            pad = num_points - sharp_points.shape[0]
            sharp_points = np.pad(sharp_points, ((0, pad), (0, 0)), mode='wrap')
            sharp_normals = np.pad(sharp_normals, ((0, pad), (0, 0)), mode='wrap')

        centroid = np.mean(points, axis=0)
        points -= centroid
        max_distance = np.max(np.linalg.norm(points, axis=1))
        if max_distance > 1e-6:
            points /= max_distance
            sharp_points -= centroid
            sharp_points /= max_distance

        point_cloud = np.hstack([points, normals])
        sharp_cloud = np.hstack([sharp_points, sharp_normals]) if len(sharp_points) > 0 else None

        return {
            "surface": torch.tensor(point_cloud, dtype=dtype).unsqueeze(0),
            "sharp_surface": torch.tensor(sharp_cloud, dtype=dtype).unsqueeze(0) if sharp_cloud is not None else None
        }
    except Exception as e:
        logger.error(f"Error processing {mesh_path}: {str(e)}")
        logger.debug(traceback.format_exc())
        return None

def process_all_meshes(download_dir="downloads", metadata_file="downloads/metadata.json"):
    processed_data = []
    with open(metadata_file, 'r', encoding='utf-8') as f:
        metadata = json.load(f)

    for uid, info in tqdm(metadata.items()):
        mesh_path = os.path.join(download_dir, f"{uid}.glb")
        caption = info.get('caption', '')
        image_url = info.get('thumbnails', {}).get('images', [{}])[0].get('url', '')

        if not os.path.isfile(mesh_path):
            logger.warning(f"Mesh file not found: {mesh_path}")
            continue

        mesh_inputs = process_mesh_to_vae_input(mesh_path, num_points=32768)
        if mesh_inputs is None:
            logger.warning(f"Skipping mesh due to processing error: {uid}")
            continue

        mesh_inputs = {k: v.to(device) if v is not None else None for k, v in mesh_inputs.items()}

        with torch.no_grad():
            encode_output = vae.encode(**mesh_inputs)
            if isinstance(encode_output, tuple):
                encode_output = encode_output[0]
            if hasattr(encode_output, "latent_dist"):
                latent = encode_output.latent_dist.sample()
            elif hasattr(encode_output, "sample"):
                latent = encode_output.sample()
            elif isinstance(encode_output, torch.Tensor):
                latent = encode_output
            else:
                raise RuntimeError(f"Unexpected encode_output type: {type(encode_output)}")

        tokens = tokenizer(
            caption,
            padding="max_length",
            max_length=2048,
            truncation=True,
            return_tensors="pt"
        )

        image_latent = get_image_latent(image_url, image_vae)

        processed_data.append({
            "uid": uid,
            "latent": latent.cpu().squeeze(0),
            "image_latent": image_latent.cpu().squeeze(0),
            "input_ids": tokens["input_ids"].squeeze(0),
            "attention_mask": tokens["attention_mask"].squeeze(0)
        })

        print(f"Processed {uid} — mesh shape: {mesh_inputs['surface'].shape} — caption: {caption[:60]}...")

    print(f"Successfully processed {len(processed_data)} samples")

process_all_meshes()

  0%|          | 0/100 [00:00<?, ?it/s]


NameError: name 'vae' is not defined

In [ ]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
from diffusers import AutoencoderKL
from diffusion_model import GaussianDiffusion, ModelMeanType, ModelVarType, LossType, get_named_beta_schedule
from timm.models.vision_transformer import PatchEmbed
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

# --- Config ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


image_size = 512
latent_downscale = 8
latent_size = image_size // latent_downscale  # 64
patch_size = 4
patch_dim = 4 * patch_size * patch_size      # 64
embed_dim = 1024
batch_size = 8

vae = AutoencoderKL.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder="vae").to(device).eval()
betas = get_named_beta_schedule("linear", 1000)
diffusion = GaussianDiffusion(betas=betas, model_mean_type=ModelMeanType.EPSILON, model_var_type=ModelVarType.FIXED_SMALL, loss_type=LossType.MSE)
patch_embed = PatchEmbed(img_size=latent_size, patch_size=patch_size, in_chans=4, embed_dim=embed_dim).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token


transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])



def preprocess(example):
    # Image -> latent
    image = transform(example['image'].convert("RGB")).unsqueeze(0).to(device)
    with torch.no_grad():
        latents = vae.encode(image).latent_dist.sample() * vae.config.scaling_factor
        latents = F.interpolate(latents, size=(latent_size, latent_size), mode='bilinear')
    # Add noise
    t = torch.randint(0, 1000, (1,), device=device).long()
    noise = torch.randn_like(latents)
    x_noisy = diffusion.q_sample(x_start=latents, t=t, noise=noise)
    # Flatten patches via unfold: [1,4,64,64] -> [1,256,64]
    patches = F.unfold(x_noisy, kernel_size=patch_size, stride=patch_size)  # [1,64,256]
    patches = patches.permute(0, 2, 1).squeeze(0)  # [256,64]
    # Tokenize text
    tokens = tokenizer(example['text'], padding='max_length', truncation=True,
                       max_length=2048, return_tensors='pt')
    return {
        'patch_embeddings': patches.cpu(),
        'noise': noise.squeeze(0).cpu(),
        'timestep': t.item(),
        'input_ids': tokens['input_ids'].squeeze(0),
        'attention_mask': tokens['attention_mask'].squeeze(0)
    }




dataset = load_dataset("iamkaikai/GAME-MAP-ART", split="train").select(range(10))
processed = [preprocess(ex) for ex in dataset]



# Dataset and DataLoader
class DiTDataset(torch.utils.data.Dataset):
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

def collate_fn(batch):
    return {
        'patch_embeddings': torch.stack([x['patch_embeddings'] for x in batch]),
        'noise': torch.stack([x['noise'] for x in batch]),
        'timestep': torch.tensor([x['timestep'] for x in batch], dtype=torch.long),
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
    }

dataloader = DataLoader(DiTDataset(processed), batch_size=batch_size,
                        shuffle=True, num_workers=4, collate_fn=collate_fn)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/318 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/67.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1101 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
import torch
from torch import nn
from transformers import AutoTokenizer
from mmfreelm.models.hgrn_bit.ternary_dit import TimestepEmbedder, DiTBlock, TextEmbedder, FinalLayer, DiT_models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DiT_models['DiT-S/8'](
    input_size=32,
    in_channels=4
).to(device)

model.train()

x = torch.randn(2, 4, 32, 32).to(device)  # Latent
t = torch.randint(0, 1000, (2,)).to(device)  # Timesteps
y = ["A cat", "A dog"]  # Texts

# Forward pass
output = model(x, t, y)
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")  # Should be [2, 8, 32, 32]

# Training step
target = torch.randn_like(output)
loss = nn.MSELoss()(output, target)
loss.backward()
print(f"Training step completed! Loss: {loss.item():.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Input shape: torch.Size([2, 4, 32, 32])
Output shape: torch.Size([2, 8, 32, 32])
Training step completed! Loss: 1.0211


In [ ]:
"""
A minimal training script for DiT using PyTorch DDP.
"""
import torch
# the first flag below was False when we tested this script but True makes A100 training a lot faster:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from torchvision.datasets import ImageFolder
from torchvision import transforms
import numpy as np
from collections import OrderedDict
from PIL import Image
from copy import deepcopy
from glob import glob
from time import time
import argparse
import logging
import os

from mmfreelm.models.hgrn_bit.ternary_dit import DiT_models
from diffusion import create_diffusion
from diffusers.models import AutoencoderKL


#################################################################################
#                             Training Helper Functions                         #
#################################################################################

@torch.no_grad()
def update_ema(ema_model, model, decay=0.9999):
    """
    Step the EMA model towards the current model.
    """
    ema_params = OrderedDict(ema_model.named_parameters())
    model_params = OrderedDict(model.named_parameters())

    for name, param in model_params.items():
        # TODO: Consider applying only to params that require_grad to avoid small numerical changes of pos_embed
        ema_params[name].mul_(decay).add_(param.data, alpha=1 - decay)


def requires_grad(model, flag=True):
    """
    Set requires_grad flag for all parameters in a model.
    """
    for p in model.parameters():
        p.requires_grad = flag


def cleanup():
    """
    End DDP training.
    """
    dist.destroy_process_group()


def create_logger(logging_dir):
    """
    Create a logger that writes to a log file and stdout.
    """
    if dist.get_rank() == 0:  # real logger
        logging.basicConfig(
            level=logging.INFO,
            format='[\033[34m%(asctime)s\033[0m] %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S',
            handlers=[logging.StreamHandler(), logging.FileHandler(f"{logging_dir}/log.txt")]
        )
        logger = logging.getLogger(__name__)
    else:  # dummy logger (does nothing)
        logger = logging.getLogger(__name__)
        logger.addHandler(logging.NullHandler())
    return logger


def center_crop_arr(pil_image, image_size):
    """
    Center cropping implementation from ADM.
    https://github.com/openai/guided-diffusion/blob/8fb3ad9197f16bbc40620447b2742e13458d2831/guided_diffusion/image_datasets.py#L126
    """
    while min(*pil_image.size) >= 2 * image_size:
        pil_image = pil_image.resize(
            tuple(x // 2 for x in pil_image.size), resample=Image.BOX
        )

    scale = image_size / min(*pil_image.size)
    pil_image = pil_image.resize(
        tuple(round(x * scale) for x in pil_image.size), resample=Image.BICUBIC
    )

    arr = np.array(pil_image)
    crop_y = (arr.shape[0] - image_size) // 2
    crop_x = (arr.shape[1] - image_size) // 2
    return Image.fromarray(arr[crop_y: crop_y + image_size, crop_x: crop_x + image_size])


#################################################################################
#                                  Training Loop                                #
#################################################################################

def main(args):
    """
    Trains a new DiT model.
    """
    assert torch.cuda.is_available(), "Training currently requires at least one GPU."

    # Setup DDP:
    dist.init_process_group("nccl")
    assert args.global_batch_size % dist.get_world_size() == 0, f"Batch size must be divisible by world size."
    rank = dist.get_rank()
    device = rank % torch.cuda.device_count()
    seed = args.global_seed * dist.get_world_size() + rank
    torch.manual_seed(seed)
    torch.cuda.set_device(device)
    print(f"Starting rank={rank}, seed={seed}, world_size={dist.get_world_size()}.")

    # Setup an experiment folder:
    if rank == 0:
        os.makedirs(args.results_dir, exist_ok=True)  # Make results folder (holds all experiment subfolders)
        experiment_index = len(glob(f"{args.results_dir}/*"))
        model_string_name = args.model.replace("/", "-")  # e.g., DiT-XL/2 --> DiT-XL-2 (for naming folders)
        experiment_dir = f"{args.results_dir}/{experiment_index:03d}-{model_string_name}"  # Create an experiment folder
        checkpoint_dir = f"{experiment_dir}/checkpoints"  # Stores saved model checkpoints
        os.makedirs(checkpoint_dir, exist_ok=True)
        logger = create_logger(experiment_dir)
        logger.info(f"Experiment directory created at {experiment_dir}")
    else:
        logger = create_logger(None)

    # Create model:
    assert args.image_size % 8 == 0, "Image size must be divisible by 8 (for the VAE encoder)."
    latent_size = args.image_size // 8
    model = DiT_models[args.model](
        input_size=latent_size,
        num_classes=args.num_classes
    )
    # Note that parameter initialization is done within the DiT constructor
    ema = deepcopy(model).to(device)  # Create an EMA of the model for use after training
    requires_grad(ema, False)
    model = DDP(model.to(device), device_ids=[rank])
    diffusion = create_diffusion(timestep_respacing="")  # default: 1000 steps, linear noise schedule
    vae = AutoencoderKL.from_pretrained(f"stabilityai/sd-vae-ft-{args.vae}").to(device)
    logger.info(f"DiT Parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Setup optimizer (we used default Adam betas=(0.9, 0.999) and a constant learning rate of 1e-4 in our paper):
    opt = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0)

    # Setup data:
    transform = transforms.Compose([
        transforms.Lambda(lambda pil_image: center_crop_arr(pil_image, args.image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=True)
    ])
    dataset = ImageFolder(args.data_path, transform=transform)
    sampler = DistributedSampler(
        dataset,
        num_replicas=dist.get_world_size(),
        rank=rank,
        shuffle=True,
        seed=args.global_seed
    )
    loader = DataLoader(
        dataset,
        batch_size=int(args.global_batch_size // dist.get_world_size()),
        shuffle=False,
        sampler=sampler,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True
    )
    logger.info(f"Dataset contains {len(dataset):,} images ({args.data_path})")

    # Prepare models for training:
    update_ema(ema, model.module, decay=0)  # Ensure EMA is initialized with synced weights
    model.train()  # important! This enables embedding dropout for classifier-free guidance
    ema.eval()  # EMA model should always be in eval mode

    # Variables for monitoring/logging purposes:
    train_steps = 0
    log_steps = 0
    running_loss = 0
    start_time = time()

    logger.info(f"Training for {args.epochs} epochs...")
    for epoch in range(args.epochs):
        sampler.set_epoch(epoch)
        logger.info(f"Beginning epoch {epoch}...")
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            with torch.no_grad():
                # Map input images to latent space + normalize latents:
                x = vae.encode(x).latent_dist.sample().mul_(0.18215)
            t = torch.randint(0, diffusion.num_timesteps, (x.shape[0],), device=device)
            model_kwargs = dict(y=y)
            loss_dict = diffusion.training_losses(model, x, t, model_kwargs)
            loss = loss_dict["loss"].mean()
            opt.zero_grad()
            loss.backward()
            opt.step()
            update_ema(ema, model.module)

            # Log loss values:
            running_loss += loss.item()
            log_steps += 1
            train_steps += 1
            if train_steps % args.log_every == 0:
                # Measure training speed:
                torch.cuda.synchronize()
                end_time = time()
                steps_per_sec = log_steps / (end_time - start_time)
                # Reduce loss history over all processes:
                avg_loss = torch.tensor(running_loss / log_steps, device=device)
                dist.all_reduce(avg_loss, op=dist.ReduceOp.SUM)
                avg_loss = avg_loss.item() / dist.get_world_size()
                logger.info(f"(step={train_steps:07d}) Train Loss: {avg_loss:.4f}, Train Steps/Sec: {steps_per_sec:.2f}")
                # Reset monitoring variables:
                running_loss = 0
                log_steps = 0
                start_time = time()

            # Save DiT checkpoint:
            if train_steps % args.ckpt_every == 0 and train_steps > 0:
                if rank == 0:
                    checkpoint = {
                        "model": model.module.state_dict(),
                        "ema": ema.state_dict(),
                        "opt": opt.state_dict(),
                        "args": args
                    }
                    checkpoint_path = f"{checkpoint_dir}/{train_steps:07d}.pt"
                    torch.save(checkpoint, checkpoint_path)
                    logger.info(f"Saved checkpoint to {checkpoint_path}")
                dist.barrier()

    model.eval()  # important! This disables randomized embedding dropout
    # do any sampling/FID calculation/etc. with ema (or model) in eval mode ...

    logger.info("Done!")
    cleanup()


if __name__ == "__main__":
    # Default args here will train DiT-XL/2 with the hyperparameters we used in our paper (except training iters).
    parser = argparse.ArgumentParser()
    parser.add_argument("--data-path", type=str, required=True)
    parser.add_argument("--results-dir", type=str, default="results")
    parser.add_argument("--model", type=str, choices=list(DiT_models.keys()), default="DiT-XL/2")
    parser.add_argument("--image-size", type=int, choices=[256, 512], default=256)
    parser.add_argument("--num-classes", type=int, default=1000)
    parser.add_argument("--epochs", type=int, default=1400)
    parser.add_argument("--global-batch-size", type=int, default=256)
    parser.add_argument("--global-seed", type=int, default=0)
    parser.add_argument("--vae", type=str, choices=["ema", "mse"], default="ema")  # Choice doesn't affect training
    parser.add_argument("--num-workers", type=int, default=4)
    parser.add_argument("--log-every", type=int, default=100)
    parser.add_argument("--ckpt-every", type=int, default=50_000)
    args = parser.parse_args()
    main(args)

3D GENERATION MODEL + 3D VAE

Additional Snippet

In [ ]:
import torch as th
import numpy as np
from diffusion_model import GaussianDiffusion, ModelMeanType, ModelVarType, LossType, get_named_beta_schedule
from diffusers import AutoencoderKL
import matplotlib.pyplot as plt

device = "cuda" if th.cuda.is_available() else "cpu"
vae = AutoencoderKL.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder="vae").to(device)
vae.eval()

num_timesteps = 1000
betas = get_named_beta_schedule("linear", num_timesteps)
diffusion = GaussianDiffusion(
    betas=betas,
    model_mean_type=ModelMeanType.EPSILON,
    model_var_type=ModelVarType.FIXED_SMALL,
    loss_type=LossType.MSE
)

batch_size, channels, height, width = 8, 3, 256, 256
x_start = th.randn((batch_size, channels, height, width), device=device)
x_start = th.clamp(x_start, -1, 1)
with th.no_grad():
    latent_dist = vae.encode(x_start).latent_dist
    latent = latent_dist.sample() * vae.config.scaling_factor
t = th.tensor([500] * batch_size, device=device, dtype=th.long)
noise = th.randn_like(latent)
x_noisy = diffusion.q_sample(x_start=latent, t=t, noise=noise)


In [ ]:
import torch
import numpy as np
from diffusion_model import GaussianDiffusion, ModelMeanType, ModelVarType, LossType, get_named_beta_schedule
import patch_embedding
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_timesteps = 1000
betas = get_named_beta_schedule("linear", num_timesteps)
diffusion = GaussianDiffusion(
    betas=betas,
    model_mean_type=ModelMeanType.EPSILON,
    model_var_type=ModelVarType.FIXED_SMALL,
    loss_type=LossType.MSE
)

batch_size, channels, height, width = 8, 3, 256, 256
x_start = torch.randn((batch_size, channels, height, width), device=device)

x_start = torch.clamp(x_start, -1, 1)

t = torch.tensor([500] * batch_size, device=device, dtype=torch.long)
noise = torch.randn_like(x_start)
x_noisy = diffusion.q_sample(x_start=x_start, t=t, noise=noise)

resize_transform = transforms.Resize((224, 224))
x_noisy_resized = resize_transform(x_noisy)

embeddings = patch_embedding.process_noised_image(x_noisy_resized, img_size=224, patch_size=14, embed_dim=768)

print(f"Original image shape: {x_start.shape}")
print(f"Noisy image shape: {x_noisy.shape}")
print(f"Resized noisy image shape: {x_noisy_resized.shape}")
print(f"Sample original values (first image, first channel, top-left 2x2):\n{x_start[0, 0, :2, :2]}")
print(f"Sample noisy values (first image, first channel, top-left 2x2):\n{x_noisy_resized[0, 0, :2, :2]}")
print(f"Patch embeddings shape: {embeddings.shape}")

3D REPRESENTATION [Point Cloud, Voxel, Mesh, Implicit Fields,ETC]